In [47]:
import ee
import geopandas as gpd
import fiona
import shapely
import contextily as cx
import json
from src.visualization import ee_viz
from src.data import mtbs
from src.data import land_cover
from src.data import shape_processor
import src.data.ee_utils as ee_utils
import numpy as np
from src.data import terrain

import importlib
importlib.reload(ee_viz)
importlib.reload(ee_utils)
importlib.reload(mtbs)
importlib.reload(terrain)

2023-05-25 10:49:53,136 DEBUG: Logger /home/jk871/fire-regen/src/data/ee_utils.py already set up. [in get_logger at /home/jk871/fire-regen/src/utils/logging_util.py:51]


<module 'src.data.terrain' from '/home/jk871/fire-regen/src/data/terrain.py'>

In [3]:
ee.Authenticate()


Successfully saved authorization token.


In [2]:
ee.Initialize()

### Get Regions of interest

In [5]:
# Fetch simplified regions of interest - to avoid processing problems with really complicated shapes.
seki = gpd.read_file("../../data/shapefiles/seki_convex_hull.shp")
sierras = gpd.read_file("../../data/shapefiles/sierras_convex_hull.shp")

In [6]:
seki_complex = gpd.read_file("../../data/shapefiles/seki.shp")
seki_box = shape_processor.get_box(seki_complex)

In [7]:
sierras_complex = gpd.read_file("../../data/shapefiles/sierras.shp")
sierras_box = shape_processor.get_box(sierras_complex)

In [ ]:
seki.to_crs(3857).explore()

In [44]:
sierras.to_crs(3857).explore()

### Get Burn Severity Data

In [12]:
burn_severity_ic = mtbs.get_burn_severity_data().sort('system:time_start', False).filterDate('2020', '2021')

In [13]:
burn_severity_ic

In [14]:
ee_viz.viz_burn_severity(burn_severity_ic.first(), sierras.geometry.iloc[0], ['burn_severity', 'burn_year'])

{'max': {'burn_severity': 6, 'burn_year': 2020}, 'mean': {'burn_severity': 2.5847411171975057, 'burn_year': 2020}, 'min': {'burn_severity': 1, 'burn_year': 2020}, 'std': {'burn_severity': 0.9584508443717643, 'burn_year': 0}, 'sum': {'burn_severity': 14269359.91372549, 'burn_year': 11151641777.176472}}


{'max': {'burn_severity': 6, 'burn_year': 2020}, 'mean': {'burn_severity': 2.5847411171975057, 'burn_year': 2020}, 'min': {'burn_severity': 1, 'burn_year': 2020}, 'std': {'burn_severity': 0.9584508443717643, 'burn_year': 0}, 'sum': {'burn_severity': 14269359.91372549, 'burn_year': 11151641777.176472}}


Map(center=[38.81702351908157, -119.91791513913883], controls=(WidgetControl(options=['position', 'transparent…

### Get aggregated burn data - last burn severity for each pixel, and burn count for each pixel

In [6]:
burn_data = mtbs.get_aggregated_burn_data()
burn_data

In [ ]:
ee_viz.viz_burn_severity(burn_data, seki.geometry.iloc[0], ['last_burn_severity', 'last_burn_year', 'burn_count'])

In [8]:
ee_viz.viz_burn_severity(burn_data, sierras_box.geometry.iloc[0], ['last_burn_severity', 'last_burn_year', 'burn_count'])

Map(center=[38.529900913720695, -119.99158443785402], controls=(WidgetControl(options=['position', 'transparen…

### Save Fire Raster as Geo Tiff

In [ ]:
ee_utils.save_image_to_drive(burn_data, polygon=seki_box.geometry[0], img_name='burn_data', scale=30)

In [ ]:
ee_utils.save_image_to_drive(burn_data, polygon=sierras_box.geometry[0], img_name='burn_data_sierras', scale=30)

### Get Landsat data

In [35]:
from src.data.ee import landsat_import
importlib.reload(landsat_import)
importlib.reload(ee_utils)

2023-05-25 10:38:36,224 DEBUG: Logger /home/jk871/fire-regen/src/data/ee_utils.py already set up. [in get_logger at /home/jk871/fire-regen/src/utils/logging_util.py:51]


<module 'src.data.ee_utils' from '/home/jk871/fire-regen/src/data/ee_utils.py'>

In [24]:
ls8_2018 = landsat_import.get_landsat(sierras.geometry[0], '2019-01-01')
ls8_2019 = landsat_import.get_landsat(sierras.geometry[0], '2020-01-01')
ls8_2020 = landsat_import.get_landsat(sierras.geometry[0], '2021-01-01')
ls8_2021 = landsat_import.get_landsat(sierras.geometry[0], '2022-01-01')
ls8_2022 = landsat_import.get_landsat(sierras.geometry[0], '2023-01-01')

In [32]:
ls8.bandNames().getInfo()

['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'nbr', 'ndvi']

In [ ]:
bands = ['SR']

In [36]:
ee_utils.save_image_to_drive_per_band(ls8_2018, polygon=sierras_box.geometry[0], img_name="landsat_2018", scale=30)

In [41]:
ee_utils.save_image_to_drive_per_band(ls8_2019, polygon=sierras_box.geometry[0], img_name="landsat_2019", scale=30)

In [42]:
ee_utils.save_image_to_drive_per_band(ls8_2020, polygon=sierras_box.geometry[0], img_name="landsat_2020", scale=30)

In [43]:
ee_utils.save_image_to_drive_per_band(ls8_2021, polygon=sierras_box.geometry[0], img_name="landsat_2021", scale=30)

In [44]:
ee_utils.save_image_to_drive_per_band(ls8_2022, polygon=sierras_box.geometry[0], img_name="landsat_2022", scale=30)

### Get Land Cover Raster from Dynamic World

In [45]:
dw_2018 = land_cover.get_land_cover('2018-01-01', '2019-01-01', sierras.geometry[0])
dw_2019 = land_cover.get_land_cover('2019-01-01', '2020-01-01', sierras.geometry[0])
dw_2020 = land_cover.get_land_cover('2020-01-01', '2021-01-01', sierras.geometry[0])
dw_2021 = land_cover.get_land_cover('2021-01-01', '2022-01-01', sierras.geometry[0])
dw_2022 = land_cover.get_land_cover('2022-01-01', '2023-01-01', sierras.geometry[0])

In [46]:
ee_utils.save_image_to_drive(dw_2018, polygon=sierras_box.geometry[0], img_name='dynamic_world_2018', scale=30)
ee_utils.save_image_to_drive(dw_2019, polygon=sierras_box.geometry[0], img_name='dynamic_world_2019', scale=30)
ee_utils.save_image_to_drive(dw_2020, polygon=sierras_box.geometry[0], img_name='dynamic_world_2020', scale=30)
ee_utils.save_image_to_drive(dw_2021, polygon=sierras_box.geometry[0], img_name='dynamic_world_2021', scale=30)
ee_utils.save_image_to_drive(dw_2022, polygon=sierras_box.geometry[0], img_name='dynamic_world_2022', scale=30)

### Get Terrain Data

In [48]:
terrain_img = terrain.get_terrain_image(sierras.geometry[0], 30)
terrain_img

Name,Description
elevation,Elevation


In [49]:
ee_utils.save_image_to_drive_per_band(terrain_img, polygon=sierras_box.geometry[0], img_name="terrain", scale=30)